In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import time
import json
from tqdm import tqdm

In [ ]:
driver = webdriver.Chrome()

# 타겟 웹사이트 열기
url = 'https://news.naver.com/section/101'  # 크롤링할 웹사이트 URL
driver.get(url)


# 기사 더보기 버튼 클릭해서 모든 기사 로드하기 (test로 1000번 클릭)
more_button = driver.find_element(By.XPATH, '//*[@id="newsct"]/div[5]/div/div[2]/a')
click_cnt = 0
max_click = 1000

while click_cnt < max_click:
    try:
        more_button.click()
        time.sleep(2)
        click_cnt += 1
    except Exception as e:
        print(f"기사 로드 중 에러 발생: {e}")


# 각 기사마다 제목과 url 수집
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
title_elements = soup.find_all(class_='sa_text_strong')
url_elements = soup.find_all('a', class_='sa_text_title')

article_titles = []
article_links = []

for i in range(len(title_elements)):
    title = title_elements[i].text
    link = url_elements[i]['href']
    article_titles.append(title)
    article_links.append(link)


# 각 기사 페이지로 이동하여 기사 본문 수집
articles_content = []
for link in tqdm(article_links):
    try:
        driver.get(link)
        article_body = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#dic_area'))
        )
        articles_content.append(article_body.text)
        time.sleep(1)  # 페이지 로드 대기
    except Exception as e:
        print(f"기사를 수집하는 중 오류 발생: {e}")
        continue


# 기사 내용 저장
news_data = {}
news_data['posts'] = []

for idx, content in enumerate(articles_content):
    news_data['posts'].append({
        'idx':idx,
        'title':article_titles[idx],
        'content':content
    })

file_path = './result.json'
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(json.dumps(news_data, ensure_ascii=False))


driver.quit()